In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

In [2]:
# prepare data

N = 300
x, t = datasets.make_moons(N, noise=0.3)
t = t.reshape(N, 1)

x_train, x_test, t_train, t_test = \
    train_test_split(x, t, test_size=0.2)

In [3]:
model = Sequential()
model.add(Dense(3, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [4]:
optimizer = optimizers.SGD(learning_rate=0.1)
model.compile(optimizer=optimizer, loss='binary_crossentropy',
                metrics=['accuracy'])

model.fit(x_train, t_train,
            epochs=100, batch_size=10,
            verbose=1)

Epoch 1/100
24/24 [==============================] - 1s 2ms/step - loss: 0.8587 - accuracy: 0.4667
Epoch 2/100
24/24 [==============================] - 0s 3ms/step - loss: 0.7502 - accuracy: 0.2750
Epoch 3/100
24/24 [==============================] - 0s 3ms/step - loss: 0.7195 - accuracy: 0.3375
Epoch 4/100
24/24 [==============================] - 0s 2ms/step - loss: 0.7005 - accuracy: 0.5167
Epoch 5/100
24/24 [==============================] - 0s 2ms/step - loss: 0.6823 - accuracy: 0.5167
Epoch 6/100
24/24 [==============================] - 0s 2ms/step - loss: 0.6643 - accuracy: 0.5667
Epoch 7/100
24/24 [==============================] - 0s 2ms/step - loss: 0.6453 - accuracy: 0.6292
Epoch 8/100
24/24 [==============================] - 0s 2ms/step - loss: 0.6230 - accuracy: 0.8083
Epoch 9/100
24/24 [==============================] - 0s 2ms/step - loss: 0.6017 - accuracy: 0.7917
Epoch 10/100
24/24 [==============================] - 0s 2ms/step - loss: 0.5785 - accuracy: 0.7958
Epoch 11/

In [5]:
loss, acc = model.evaluate(x_test, t_test, verbose=0)
print("test_loss:", loss, ", test_acc:", acc)

test_loss: 0.43351447582244873 , test_acc: 0.800000011920929


In [6]:
from sklearn.utils import shuffle
from keras.models import Model
from keras import losses
from keras import metrics

In [9]:
class MLP(Model):
    def __init__(self, hidden_dim, output_dim):
        super().__init__()
        self.l1 = Dense(hidden_dim, activation='sigmoid')
        self.l2 = Dense(output_dim, activation='sigmoid')

    def call(self, x):
        h = self.l1(x)
        y = self.l2(h)
        return y

In [10]:
model = MLP(3, 1)

criterion = losses.BinaryCrossentropy()
optimizer = optimizers.SGD(learning_rate=0.1)

def compute_loss(t, y):
    return criterion(t, y)

def train_step(x, t):
    with tf.GradientTape() as tape:
        preds = model(x)
        loss = compute_loss(t, preds)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

epochs = 100
batch_size = 10
n_batches = x_train.shape[0] // batch_size

for epoch in range(epochs):
    train_loss = 0.
    x_, t_ = shuffle(x_train, t_train)

    for batch in range(n_batches):
        start = batch * batch_size
        end = start + batch_size
        loss = train_step(x_[start:end], t_[start:end])
        train_loss += loss.numpy()

    print('epoch: {}, loss: {:.3}'.format(
        epoch+1,
        train_loss
    ))

epoch: 1, loss: 17.0
epoch: 2, loss: 16.4
epoch: 3, loss: 16.0
epoch: 4, loss: 15.7
epoch: 5, loss: 15.4
epoch: 6, loss: 15.0
epoch: 7, loss: 14.6
epoch: 8, loss: 14.3
epoch: 9, loss: 13.9
epoch: 10, loss: 13.4
epoch: 11, loss: 13.1
epoch: 12, loss: 12.7
epoch: 13, loss: 12.3
epoch: 14, loss: 12.0
epoch: 15, loss: 11.7
epoch: 16, loss: 11.4
epoch: 17, loss: 11.1
epoch: 18, loss: 10.9
epoch: 19, loss: 10.7
epoch: 20, loss: 10.5
epoch: 21, loss: 10.3
epoch: 22, loss: 10.1
epoch: 23, loss: 10.0
epoch: 24, loss: 9.87
epoch: 25, loss: 9.76
epoch: 26, loss: 9.66
epoch: 27, loss: 9.54
epoch: 28, loss: 9.47
epoch: 29, loss: 9.39
epoch: 30, loss: 9.32
epoch: 31, loss: 9.25
epoch: 32, loss: 9.2
epoch: 33, loss: 9.17
epoch: 34, loss: 9.09
epoch: 35, loss: 9.06
epoch: 36, loss: 9.01
epoch: 37, loss: 9.0
epoch: 38, loss: 8.95
epoch: 39, loss: 8.93
epoch: 40, loss: 8.89
epoch: 41, loss: 8.87
epoch: 42, loss: 8.85
epoch: 43, loss: 8.84
epoch: 44, loss: 8.82
epoch: 45, loss: 8.78
epoch: 46, loss: 8.78

In [11]:
test_loss = metrics.Mean()
test_acc = metrics.BinaryAccuracy()

def test_step(x, t):
    preds = model(x)
    loss = compute_loss(t, preds)
    test_loss(loss)
    test_acc(t, preds)

    return loss

test_step(x_test, t_test)

print('test_loss: {:.3f}, test_acc: {:.3f}'.format(
    test_loss.result(),
    test_acc.result()
))


test_loss: 0.432, test_acc: 0.800


In [12]:
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers

In [13]:
class MLP(nn.Module):
    '''
    多層パーセプトロン
    '''
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.l1 = nn.Linear(input_dim, hidden_dim)
        self.a1 = nn.Sigmoid()
        self.l2 = nn.Linear(hidden_dim, output_dim)
        self.a2 = nn.Sigmoid()

        self.layers = [self.l1, self.a1, self.l2, self.a2]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)

        return x

In [14]:
np.random.seed(123)
torch.manual_seed(123)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

'''
1. データの準備
'''
N = 300
x, t = datasets.make_moons(N, noise=0.3)
t = t.reshape(N, 1)

x_train, x_test, t_train, t_test = \
    train_test_split(x, t, test_size=0.2)

In [15]:
# prepare model
model = MLP(2, 3, 1).to(device)

In [16]:
criterion = nn.BCELoss()
optimizer = optimizers.SGD(model.parameters(), lr=0.1)

def compute_loss(t, y):
    return criterion(y, t)

def train_step(x, t):
    model.train()
    preds = model(x)
    loss = compute_loss(t, preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss

epochs = 100
batch_size = 10
n_batches = x_train.shape[0] // batch_size

for epoch in range(epochs):
    train_loss = 0.
    x_, t_ = shuffle(x_train, t_train)
    x_ = torch.Tensor(x_).to(device)
    t_ = torch.Tensor(t_).to(device)

    for n_batch in range(n_batches):
        start = n_batch * batch_size
        end = start + batch_size
        loss = train_step(x_[start:end], t_[start:end])
        train_loss += loss.item()

    print('epoch: {}, loss: {:.3}'.format(
        epoch+1,
        train_loss
    ))

epoch: 1, loss: 17.1
epoch: 2, loss: 16.8
epoch: 3, loss: 16.6
epoch: 4, loss: 16.4
epoch: 5, loss: 16.1
epoch: 6, loss: 15.9
epoch: 7, loss: 15.6
epoch: 8, loss: 15.3
epoch: 9, loss: 14.9
epoch: 10, loss: 14.5
epoch: 11, loss: 14.1
epoch: 12, loss: 13.7
epoch: 13, loss: 13.3
epoch: 14, loss: 12.9
epoch: 15, loss: 12.5
epoch: 16, loss: 12.2
epoch: 17, loss: 11.8
epoch: 18, loss: 11.6
epoch: 19, loss: 11.3
epoch: 20, loss: 11.1
epoch: 21, loss: 10.9
epoch: 22, loss: 10.8
epoch: 23, loss: 10.6
epoch: 24, loss: 10.5
epoch: 25, loss: 10.4
epoch: 26, loss: 10.3
epoch: 27, loss: 10.2
epoch: 28, loss: 10.1
epoch: 29, loss: 10.1
epoch: 30, loss: 10.0
epoch: 31, loss: 9.96
epoch: 32, loss: 9.92
epoch: 33, loss: 9.86
epoch: 34, loss: 9.82
epoch: 35, loss: 9.79
epoch: 36, loss: 9.75
epoch: 37, loss: 9.73
epoch: 38, loss: 9.71
epoch: 39, loss: 9.68
epoch: 40, loss: 9.64
epoch: 41, loss: 9.63
epoch: 42, loss: 9.61
epoch: 43, loss: 9.6
epoch: 44, loss: 9.56
epoch: 45, loss: 9.55
epoch: 46, loss: 9.5

In [17]:
def test_step(x, t):
    x = torch.Tensor(x).to(device)
    t = torch.Tensor(t).to(device)
    model.eval()
    preds = model(x)
    loss = compute_loss(t, preds)

    return loss, preds

loss, preds = test_step(x_test, t_test)
test_loss = loss.item()
preds = preds.data.cpu().numpy() > 0.5
test_acc = accuracy_score(t_test, preds)

print('test_loss: {:.3f}, test_acc: {:.3f}'.format(
    test_loss,
    test_acc
))

test_loss: 0.302, test_acc: 0.867
